In [1]:

import tabula
import pandas as pd


In [2]:
lista_tabelas = tabula.read_pdf('historico_joao.pdf', pages = ['all'], multiple_tables= True)
print(len(lista_tabelas))

6


In [3]:
list_columns = ['Ano/Periodo Letivo', 'Componente', 'Codigo', 'Nome', 'CH', 'Turma', 'Freq %', 'Nota', 'Situacao']

In [4]:
tabelas = []

for i in lista_tabelas:
    componente = (i.iloc[0] == 'Componente Curricular').any()
    nota = (i.iloc[0] == 'Nota').any()
    if componente and nota:
        print(len(i.columns))
        display(i.head(8))
        tabelas.append(i)
    else:
        print('Tabela não é do histórico')

9


,Ano/Período,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,NaN,NaN,NaN,Componente Curricular,CH,Turma,Freq %,Nota,Situação
1,Letivo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,--,*,CIC0004,ALGORITMOS E PROGRAMAÇÃO DE COMPUTADORES,90,--,"100,0",-,CUMP
3,--,#,CIC0007,INTRODUÇÃO À CIÊNCIA DA COMPUTAÇÃO,60,--,"100,0",-,CUMP
4,--,*,CIC0088,COMPUTACAO BASICA,90,--,"100,0",-,CUMP
5,--,#,DEG0204,VIGILÂNCIA EPIDEMIOLÓGICA COMUNITÁRIA E PARTIC...,60,--,"100,0",-,CUMP
6,--,#,ENM0068,TERMODINAMICA 1,60,--,"100,0",-,CUMP
7,--,*,ENM0080,FENOMENOS DE TRANSPORTE,75,--,"100,0",-,CUMP


9


,Ano/Período,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,NaN,NaN,NaN,Componente Curricular,CH,Turma,Freq %,Nota,Situação
1,Letivo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,--,NaN,FGA0168,DESENHO INDUSTRIAL ASSISTIDO POR COMPUTADOR,90,--,"100,0",-,CUMP
3,--,*,FGA0175,ENGENHARIA DE PETRÓLEO E GÁS,60,--,"100,0",-,CUMP
4,--,NaN,IFD0171,FISICA 1,60,--,"100,0",-,CUMP
5,--,NaN,IFD0173,FISICA 1 EXPERIMENTAL,30,--,"100,0",-,CUMP
6,--,#,ILD0036,INGLÊS INSTRUMENTAL 1,60,--,"100,0",-,CUMP
7,--,#,IQD0005,TRANSFERÊNCIA DE QUANTIDADE DE MOVIMENTO,60,--,"100,0",-,CUMP


Tabela não é do histórico
Tabela não é do histórico
Tabela não é do histórico
Tabela não é do histórico


In [5]:
situacao_aceitas = ['CUMP', 'APR', 'MATR']
colunas_fixas = ['Codigo', 'Nome']
tabelas_formatadas = []

for tabela in tabelas:
    try:
        if len(tabela.columns) == 10:
            print(len(tabela.columns))
            del tabela['Unnamed: 3']
        
        tabela.drop([0, 1], inplace=True)
        tabela.columns = list_columns
        
        tabela = tabela[tabela['Situacao'].isin(situacao_aceitas)]
        
        tabela = tabela[colunas_fixas]
        tabela.reset_index(drop=True, inplace=True)
        #display(tabela)
        tabelas_formatadas.append(tabela)
    except Exception as e:
        print('Erro: ', e)
        pass
    
tabela_final = pd.concat(tabelas_formatadas, ignore_index=True)
display(tabela_final)

,Codigo,Nome
0,CIC0004,ALGORITMOS E PROGRAMAÇÃO DE COMPUTADORES
1,CIC0007,INTRODUÇÃO À CIÊNCIA DA COMPUTAÇÃO
2,CIC0088,COMPUTACAO BASICA
3,DEG0204,VIGILÂNCIA EPIDEMIOLÓGICA COMUNITÁRIA E PARTIC...
4,ENM0068,TERMODINAMICA 1
5,ENM0080,FENOMENOS DE TRANSPORTE
6,FGA0067,TEORIA DE CIRCUITOS ELETRÔNICOS 1
7,FGA0069,PRÁTICA DE CIRCUITOS ELETRÔNICOS 1
8,FGA0078,TEORIA DE MATERIAIS DE CONSTRUÇÃO
9,FGA0102,SINAIS E SISTEMAS PARA ENGENHARIA


In [6]:
for i in tabelas:
    display(i)

,Ano/Periodo Letivo,Componente,Codigo,Nome,CH,Turma,Freq %,Nota,Situacao
2,--,*,CIC0004,ALGORITMOS E PROGRAMAÇÃO DE COMPUTADORES,90,--,"100,0",-,CUMP
3,--,#,CIC0007,INTRODUÇÃO À CIÊNCIA DA COMPUTAÇÃO,60,--,"100,0",-,CUMP
4,--,*,CIC0088,COMPUTACAO BASICA,90,--,"100,0",-,CUMP
5,--,#,DEG0204,VIGILÂNCIA EPIDEMIOLÓGICA COMUNITÁRIA E PARTIC...,60,--,"100,0",-,CUMP
6,--,#,ENM0068,TERMODINAMICA 1,60,--,"100,0",-,CUMP
7,--,*,ENM0080,FENOMENOS DE TRANSPORTE,75,--,"100,0",-,CUMP
8,--,&,FGA0067,TEORIA DE CIRCUITOS ELETRÔNICOS 1,60,--,"100,0",-,CUMP
9,--,&,FGA0069,PRÁTICA DE CIRCUITOS ELETRÔNICOS 1,30,--,"100,0",-,CUMP
10,--,*,FGA0078,TEORIA DE MATERIAIS DE CONSTRUÇÃO,45,--,"100,0",-,CUMP
11,--,&,FGA0102,SINAIS E SISTEMAS PARA ENGENHARIA,90,--,"100,0",-,CUMP


,Ano/Periodo Letivo,Componente,Codigo,Nome,CH,Turma,Freq %,Nota,Situacao
2,--,NaN,FGA0168,DESENHO INDUSTRIAL ASSISTIDO POR COMPUTADOR,90,--,"100,0",-,CUMP
3,--,*,FGA0175,ENGENHARIA DE PETRÓLEO E GÁS,60,--,"100,0",-,CUMP
4,--,NaN,IFD0171,FISICA 1,60,--,"100,0",-,CUMP
5,--,NaN,IFD0173,FISICA 1 EXPERIMENTAL,30,--,"100,0",-,CUMP
6,--,#,ILD0036,INGLÊS INSTRUMENTAL 1,60,--,"100,0",-,CUMP
7,--,#,IQD0005,TRANSFERÊNCIA DE QUANTIDADE DE MOVIMENTO,60,--,"100,0",-,CUMP
8,--,*,IQD0125,QUIMICA GERAL TEORICA,60,--,"100,0",-,CUMP
9,--,*,IQD0126,QUIMICA GERAL EXPERIMENTAL,30,--,"100,0",-,CUMP
10,--,NaN,MAT0025,CÁLCULO 1,90,--,"100,0",-,CUMP
11,--,NaN,MAT0026,CÁLCULO 2,90,--,"100,0",-,CUMP
